In [1]:
import os
import re
import warnings
import unicodedata
import json
import pickle

#instalar 

import numpy as np
from numpy.random import choice
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import nltk
from nltk.tag import pos_tag
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorboard.plugins import projector

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [3]:
filepath = 'data/'
modelpath = 'models/'

In [4]:
data = pickle.load(open(filepath+'paper_data.pkl','rb'))

In [5]:
model = Word2Vec.load(modelpath+'sgns_200.model')

In [6]:
model.wv.most_similar('COVID-19')

[('COVID-19,', 0.8055044412612915),
 ('2019-nCoV', 0.7944456338882446),
 ('SARS-CoV-2', 0.7608892917633057),
 ('COVID-19.', 0.7404037714004517),
 ('COVID-1', 0.7127674221992493),
 ('Wuhan', 0.7018991112709045),
 ('NCP', 0.6837043166160583),
 ('Wuhan.', 0.6797844171524048),
 ('2019-nCoV,', 0.6686412692070007),
 ('COVID-', 0.6644499897956848)]

In [7]:
LOG_DIR = 'sgns_proj'
variable_name = 'novar'
path_metadata = os.path.join('metadata.tsv')

In [8]:
test_size = 1000
vali_size = len(model.wv.vocab)
test = False
if test:
    size = test_size
else:
    size = vali_size

In [9]:
w2v = np.zeros((len(model.wv.vocab),model.trainables.layer1_size))

In [10]:
remove_file = True
if remove_file and os.path.isfile(path_metadata):
    os.remove(path_metadata)
if remove_file or not os.path.isfile(path_metadata):
    with open(path_metadata, 'w+') as file_metadata:
        for i in range(len(model.wv.vocab)):
            w2v[i] = model.wv[model.wv.index2word[i]]
            file_metadata.write('%s\n' % model.wv.index2word[i])

In [11]:
embedding = tf.Variable(w2v, trainable=False, name=variable_name)
summary_writer = tf.summary.FileWriter(LOG_DIR)

In [12]:
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = path_metadata
projector.visualize_embeddings(summary_writer, config)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [14]:
saver.save(sess, os.path.join(LOG_DIR,'model.ckpt'), 1)

'sgns_proj/model.ckpt-1'